In [24]:
# Importing required libraries

import time
import requests
import select as something

from bs4 import BeautifulSoup
from selenium import webdriver
from pymongo import MongoClient
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [25]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['taller2']
collection = db['elespectador']

In [26]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [27]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver'

In [28]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\eveli\AppData\Local\Temp/ipykernel_11040/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [29]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
for x in range (1,32):
    soup = make_request(browser, '/archivo/politica/')
    entrada = browser.find_element_by_class_name('Pagination-Input')
    entrada.send_keys(Keys.BACK_SPACE)
    entrada.send_keys(Keys.BACK_SPACE)
    entrada.send_keys(x)
    entrada.send_keys(Keys.ENTER)
    time.sleep(15)
    
# Finding the section where news are contained 
    layout = soup.find(class_ = 'Layout-flexAds')

# Getting blocks from layout
    blocks = layout.find_all(class_ = 'Container Block', recursive = True)
    print(len(blocks))
    
# Finding and concatenating news cards
    cards = blocks[0].find_all(class_ = 'Card_rowCardLeft')
    len(cards)

# Building a list with title and relative path of the news founded
    news = []

    for card in cards:
        news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href'],       
    })
    
    news
    
    for n in news:
        # Getting HTML content for each news page
        soup = make_request(browser, n['relative_path'])

        # Extracting news metadata
        n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
        n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
        n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
        n['category'] = soup.find(class_ = 'Breadcrumb').find('a').get_text()

        # Extracting and concatenating news full text
        paragraphs = soup.find_all(class_ = 'font--secondary')
        n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

    news

C:\Users\eveli\AppData\Local\Temp/ipykernel_11040/846641386.py:3: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  entrada = browser.find_element_by_class_name('Pagination-Input')


1
2
2
1
2
2
2
2
1
1
2
2
2
1
1


In [ ]:
# Storing extracted information for further analysis
collection.insert_many(news)